In [2]:
from bs4 import BeautifulSoup
import requests as r
import pandas as pd

In [3]:
source = r.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

In [12]:
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

# Create list first then convert it into a df
data = []

table = soup.find("table")

for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row = td_cell.text.strip()
        
        # Postal value
        postal = row[0:3]
        i = row.find("(")
        
        # if neighbourhood exist
        if i != -1:
            # Borough value
            borough = row[3:i]
            neigh_list = row[i+1:]
  

            neigh = ""
            
            for i in range(0, len(neigh_list)):
                
                if neigh_list[i] == "/" or neigh_list[i] == ")":
                    neigh = neigh.lstrip()
                    row_data = [postal, borough, neigh]
                    data.append(row_data)
                    row_data = []
                    neigh = ""
                        
                else:
                    neigh += neigh_list[i]
                        
                
        else:
            # Add postal and borough value to data
            borough = row[3:]
            row_data = [postal, borough]
            data.append(row_data)
            


# Convert dataset into dataframe
df = pd.DataFrame(data, columns = column_names)



In [13]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,None
1,M2A,Not assigned,None
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park


In [14]:
# Remove Not Assigned Borough and None Neighborhood

df = df[df.Borough != 'Not assigned']
df = df.groupby(['Postalcode', 'Borough'], sort=False).agg(', '.join)
df.reset_index(inplace=True)
df.head()


,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [15]:
df.shape

(103, 3)